In [1]:
import os
import rasterio  as ras
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

from rasterio.warp import calculate_default_transform, reproject, Resampling


In [2]:
# Define the directory containing .tif files (converted from jp2 to tiffs in 3_0)
input_dop_folder = "data/dop_jp2_to_tiffs"
output_dop_folder = "data/dop_resampled_crs_Normalize"

os.makedirs(output_dop_folder, exist_ok=True)  # Ensure the output folder exists


In [3]:
# Get a sorted list of .tif files in the folder
all_files = [f for f in os.listdir(input_dop_folder ) if f.endswith((".tif", ".tiff"))]

# Select the first 4 files (or a specific slice)
files_to_check = all_files[0:4]

# Loop through the selected files and display their resolutions
for filename in files_to_check:
    file_path = os.path.join(input_dop_folder, filename)
    
    # Open the file and extract resolution
    with ras.open(file_path) as src:
        resolution = src.res  # Get resolution (pixel size in x and y)
        print(f"File: {filename} - Resolution (X, Y): {resolution}")

File: dop10rgbi_32_280_5652_1_nw_2023.tif - Resolution (X, Y): (0.1, 0.1)
File: dop10rgbi_32_280_5653_1_nw_2023.tif - Resolution (X, Y): (0.1, 0.1)
File: dop10rgbi_32_280_5654_1_nw_2023.tif - Resolution (X, Y): (0.1, 0.1)
File: dop10rgbi_32_280_5655_1_nw_2023.tif - Resolution (X, Y): (0.1, 0.1)


In [4]:

# Target CRS (SpaceNet 6) and resolution
target_crs = "EPSG:4326"
target_resolution = 0.5  # SpaceNet standard resolution (0.5m per pixel)

# Normalization range
normalize_to = [0, 1]  # Set to [0, 255] if required by the model

In [8]:


def normalize_band(band, range_min, range_max):
    """
    Normalize a raster band to the specified range.
    """
    band_min = band.min()
    band_max = band.max()
    normalized_band = (band - band_min) / (band_max - band_min) * (range_max - range_min) + range_min
    return normalized_band

# Process each .tif file
for filename in os.listdir(input_dop_folder):
    if filename.endswith(".tif"):
        input_path = os.path.join(input_dop_folder, filename)
        output_path = os.path.join(output_dop_folder, filename)

        #input_dop_folder
        #output_dop_folder 

        with ras.open(input_path) as src:
            # Step 1: Normalize pixel values
            profile = src.profile
            normalized_bands = []
            for i in range(1, src.count + 1):
                band = src.read(i).astype("float32")  # Read band data as float32 for normalization
                normalized_band = normalize_band(band, normalize_to[0], normalize_to[1])
                normalized_bands.append(normalized_band)

            # Step 2: Calculate resampling dimensions
            scale_x = src.res[0] / target_resolution
            scale_y = src.res[1] / target_resolution
            new_width = int(src.width / scale_x)
            new_height = int(src.height / scale_y)
            resampled_transform = src.transform * src.transform.scale(scale_x, scale_y)

            # Step 3: Calculate reprojection dimensions
            reproj_transform, reproj_width, reproj_height = calculate_default_transform(
                src.crs, target_crs, new_width, new_height, *src.bounds
            )

            # Update metadata for reprojection and resampling
            profile.update(
                crs=target_crs,
                transform=reproj_transform,
                width=reproj_width,
                height=reproj_height,
                dtype="float32"
            )

            # Step 4: Resample and reproject
            with ras.open(output_path, "w", **profile) as dst:
                for i, normalized_band in enumerate(normalized_bands, start=1):
                    resampled_band = ras.warp.reproject(
                        source=normalized_band,
                        destination=ras.band(dst, i),
                        src_transform=resampled_transform,
                        src_crs=src.crs,
                        dst_transform=reproj_transform,
                        dst_crs=target_crs,
                        resampling=Resampling.bilinear  # Use bilinear resampling
                    )

        print(f"Processed {input_path} -> {output_path}")

print("Batch processing complete!")


Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5652_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5652_1_nw_2023.tif
Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5653_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5653_1_nw_2023.tif
Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5654_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5654_1_nw_2023.tif
Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5655_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5655_1_nw_2023.tif
Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5656_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5656_1_nw_2023.tif
Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5657_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5657_1_nw_2023.tif
Processed data/dop_jp2_to_tiffs\dop10rgbi_32_280_5658_1_nw_2023.tif -> data/dop_resampled_crs_Normalize\dop10rgbi_32_280_5658_1_nw_2023.tif
Processed data/dop_j

In [ ]:
# Target resolution (SpaceNet standard: 0.5m per pixel)
target_resolution = 0.5

# Loop through all .tif files in the input folder
for filename in os.listdir(input_dop_folder):
    if filename.endswith((".tif", "tiff")):  # Process only .tif files
        input_path = os.path.join(input_dop_folder, filename)
        output_path = os.path.join(output_dop_folder , filename)

        with ras.open(input_path) as src:
            # Calculate new dimensions
            scale_x = src.res[0] / target_resolution
            scale_y = src.res[1] / target_resolution
            new_width = int(src.width / scale_x)
            new_height = int(src.height / scale_y)
            new_transform = src.transform * src.transform.scale(scale_x, scale_y)

            # Update metadata for the new resolution
            profile = src.profile
            profile.update(transform=new_transform, width=new_width, height=new_height)

            # Resample the data to the target resolution
            with ras.open(output_path, "w", **profile) as dst:
                for i in range(1, src.count + 1):  # Loop through all bands
                    resampled_data = src.read(
                        i,
                        out_shape=(new_height, new_width),
                        resampling=Resampling.bilinear,  # Use bilinear resampling
                    )
                    dst.write(resampled_data, i)

        print(f"Resampled {input_path} to {output_path} with resolution {target_resolution}m per pixel")

print("Batch resampling complete!")


Resampled data/dop_jp2_to_tiffs\dop10rgbi_32_280_5652_1_nw_2023.tif to data/dop_resampled\dop10rgbi_32_280_5652_1_nw_2023.tif with resolution 0.5m per pixel
Resampled data/dop_jp2_to_tiffs\dop10rgbi_32_280_5653_1_nw_2023.tif to data/dop_resampled\dop10rgbi_32_280_5653_1_nw_2023.tif with resolution 0.5m per pixel
Resampled data/dop_jp2_to_tiffs\dop10rgbi_32_280_5654_1_nw_2023.tif to data/dop_resampled\dop10rgbi_32_280_5654_1_nw_2023.tif with resolution 0.5m per pixel
Resampled data/dop_jp2_to_tiffs\dop10rgbi_32_280_5655_1_nw_2023.tif to data/dop_resampled\dop10rgbi_32_280_5655_1_nw_2023.tif with resolution 0.5m per pixel
Resampled data/dop_jp2_to_tiffs\dop10rgbi_32_280_5656_1_nw_2023.tif to data/dop_resampled\dop10rgbi_32_280_5656_1_nw_2023.tif with resolution 0.5m per pixel
